In [1]:
from __future__ import print_function

#%% import public modules
import time
import matplotlib.pyplot as plt
import copy
import autograd.numpy as np
import scipy.io as sio
import scipy.signal as sig

#%% modules for interactivity
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

#%% import custom modules
from assets.functions import FFT, IFFT, P, PSD, RFSpectrum
from classes.environment import OpticalField, OpticalField_CW, OpticalField_Pulse, OpticalField_PPLN
from classes.components import Fiber, PhaseModulator, WaveShaper, PowerSplitter, FrequencySplitter, DelayLine
from classes.experiment import Experiment

In [2]:
def plot_field(field, env, ax):
    for axi in ax:
        axi.cla()
        
    # calculate power in temporal & spectral domains & plots (normalized)
    p0 = P(env.field0)
    p = P(field) / max(p0)
    p0 *= 1 / max(p0)

    psd0 = PSD(env.field0, env.dt, env.df)
    psd = PSD(field, env.dt, env.df) /max(psd0)
    psd0 *= 1 / max(psd0)

    ax[0].plot(env.t/1e-12, p, label='Field Out', alpha=0.5)
    ax[0].plot(env.t/1e-12, p0, label='Field In', alpha=0.5)
    ax[0].set(xlabel='Time (ps)', ylabel='AU')
    ax[0].set(xlim=[-50, 10])
#     ax[0].margins(x=-0.49, y=0.05)  

    ax[1].plot(env.c0/(env.f + env.f0)/1e-6, psd, label='Field Out', alpha=0.5)
    ax[1].plot(env.c0/(env.f + env.f0)/1e-6, psd0, label='Field In', alpha=0.5)

    ax[1].set(xlabel=r'Wavelength ($\mu$m)', ylabel='AU')
#     ax[1].margins(x=-0.4, y=0.05)  
    ax[1].set(xlim=[1.52, 1.58])
    for axi in ax:
        axi.legend()
    plt.show()
    
def interactive_simulation(**kwargs):
    at = exp.newattributes()
    for i, (node, param, new_value) in enumerate(zip(nodes, params, kwargs.values())):
        at[node][param] = new_value
    print(at)
    
    exp.setattributes(at)
    exp.inject_optical_field(env.field0)
    exp.simulate(env)
    field = exp.nodes[exp.measurement_nodes[0]]['output']
    fig, ax = plt.subplots(2,1)

    plot_field(field, env, ax)
    return 

def setup_interactivity(exp):    
    up, low, name, dsct = [], [], [], []
    nodes, params = [], []
    for node in exp.nodes():
        for param in range(exp.nodes[node]['info'].N_PARAMETERS):
            up.append(exp.nodes[node]['info'].UPPER[param])
            low.append(exp.nodes[node]['info'].LOWER[param])
            name.append(exp.nodes[node]['info'].AT_NAME[param])
            dsct.append(exp.nodes[node]['info'].DSCRTVAL[param])
            nodes.append(node)
            params.append(param)
    
    kwargs = {}
    for i in range(len(up)):
        kwargs[name[i]] = FloatSlider(
            value=up[i],
            min=low[i],
            max=up[i],
            step=dsct[i],
            description=name[i],
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=False,
            readout_format='.1f',
        )
    return kwargs, nodes, params, up, low, name, dsct

In [3]:
manual = True

env = OpticalField_PPLN(n_samples=2**16, window_t=2.0e-9, lambda0=1.55e-6, bandwidth=[1.53e-6, 1.57e-6])
#env = OpticalField_CW(n_samples=2**14, window_t=10e-9, peak_power=1, lambda0=1.55e-6, normalize=False)

# %% set-up the system
components = {0: DelayLine()}
adjacency = []

# %% initialize the experiment, and perform all the pre-processing steps
exp = Experiment()
exp.buildexperiment(components, adjacency)
exp.make_path()

kwargs, nodes, params, up, low, name, dsct = setup_interactivity(exp)

if manual == True:
    interact_manual(interactive_simulation, **kwargs)
else:
    interact(interactive_simulation, **kwargs)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Delay 0', max=1.0, readout=…